[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/evals/openai-embeddings-eval.ipynb)

[![View Article](https://img.shields.io/badge/View%20Article-blue)](https://www.mongodb.com/developer/products/atlas/choose-embedding-model-rag/?utm_campaign=devrel&utm_source=cross-post&utm_medium=organic_social&utm_content=https%3A%2F%2Fgithub.com%2Fmongodb-developer%2FGenAI-Showcase&utm_term=apoorva.joshi)

# RAG Series Part 1: How to choose the right embedding model for your RAG application

This notebook evaluates the [text-embedding-3-large](https://openai.com/blog/new-embedding-models-and-api-updates) model.


## Step 1: Install required libraries

- **datasets**: Python library to get access to datasets available on Hugging Face Hub
<p>
- **openai**: Python library to interact with OpenAI APIs
<p>
- **sentence-transformers**: Framework for working with text and image embeddings
<p>
- **numpy**: Python library that provides tools to perform mathematical operations on arrays
<p>
- **pandas**: Python library for data analysis, exploration and manipulation
<p>
- **tdqm**: Python module to show a progress meter for loops


In [1]:
! pip install -qU datasets openai sentence-transformers numpy pandas tqdm

## Step 2: Setup pre-requisites

Set OpenAI API key as environment variable, and initialize the OpenAI client.

Steps to obtain an OpenAI API Key can be found [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)


In [2]:
import os
import getpass
from openai import OpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai_client = OpenAI()

OpenAI API Key:········


## Step 3: Download the evaluation dataset

We will use MongoDB's [cosmopedia-wikihow-chunked](https://huggingface.co/datasets/MongoDB/cosmopedia-wikihow-chunked) dataset, which has chunked versions of WikiHow articles from the [Cosmopedia](https://huggingface.co/datasets/HuggingFaceTB/cosmopedia) dataset released by Hugging Face. The dataset is pretty large, so we will only grab the first 25k records for testing.


In [4]:
from datasets import load_dataset
import pandas as pd

# Use streaming=True to load the dataset without downloading it fully
data = load_dataset("MongoDB/cosmopedia-wikihow-chunked", split="train", streaming=True)
# Get first 25k records from the dataset
data_head = data.take(25000)
df = pd.DataFrame(data_head)

# Use this if you want the full dataset
# data = load_dataset("AIatMongoDB/cosmopedia-wikihow-chunked", split="train")
# df = pd.DataFrame(data)

## Step 4: Data analysis

Make sure the length of the dataset is what we expect (25k), preview the data, drop Nones etc.


In [5]:
# Ensuring length of dataset is what we expect i.e. 25k
len(df)

25000

In [6]:
# Previewing the contents of the data
df.head()

,doc_id,chunk_id,text_token_length,text
0,0,0,180,Title: How to Create and Maintain a Compost Pi...
1,0,1,141,**Step 2: Gather Materials**\nGather brown (ca...
2,0,2,182,_Key guideline:_ For every volume of green mat...
3,0,3,188,_Key tip:_ Chop large items like branches and ...
4,0,4,157,**Step 7: Maturation and Use**\nAfter 3-4 mont...


In [7]:
# Only keep records where the text field is not null
df = df[df["text"].notna()]

In [8]:
# Number of unique documents in the dataset
df.doc_id.nunique()

4335

## Step 5: Creating embeddings

Define the embedding function, and run a quick test.


In [9]:
from typing import List

In [10]:
def get_embeddings(
    docs: List[str], model: str = "text-embedding-3-large"
) -> List[List[float]]:
    """
    Generate embeddings using the OpenAI API.

    Args:
        docs (List[str]): List of texts to embed
        model (str, optional): Model name. Defaults to "text-embedding-3-large".

    Returns:
        List[float]: Array of embeddings
    """
    # replace newlines, which can negatively affect performance.
    docs = [doc.replace("\n", " ") for doc in docs]
    response = openai_client.embeddings.create(input=docs, model=model)
    response = [r.embedding for r in response.data]
    return response

In [11]:
# Generating a test embedding
test_openai_embed = get_embeddings([df.iloc[0]["text"]])

In [12]:
# Sanity check to make sure embedding dimensions are as expected i.e. 3072
len(test_openai_embed[0])

3072

## Step 6: Evaluation


### Measuring embedding latency

Create a local vector store (list) of embeddings for the entire dataset.


In [13]:
from tqdm.auto import tqdm
import numpy as np

In [14]:
texts = df["text"].tolist()

In [15]:
batch_size = 128

In [16]:
embeddings = []
# Generate embeddings in batches
for i in tqdm(range(0, len(texts), batch_size)):
    end = min(len(texts), i + batch_size)
    batch = texts[i:end]
    # Generate embeddings for current batch
    batch_embeddings = get_embeddings(batch)
    # Add to the list of embeddings
    embeddings.extend(np.array(batch_embeddings))

  0%|          | 0/196 [00:00<?, ?it/s]

### Measuring retrieval quality

- Create embedding for the user query
<p>
- Get the top 5 most similar documents from the local vector store using cosine similarity as the similarity metric


In [17]:
from sentence_transformers.util import cos_sim

In [18]:
# Converting embeddings list to a Numpy array- required to calculate cosine similarity
embeddings = np.asarray(embeddings)

In [19]:
def query(query: str, top_k: int = 3) -> None:
    """
    Query the local vector store for the top 3 most relevant documents.

    Args:
        query (str): User query
        top_k (int, optional): Number of documents to return. Defaults to 3.
    """
    # Generate embedding for the user query
    query_emb = np.asarray(get_embeddings([query]))
    # Calculate cosine similarity
    scores = cos_sim(query_emb, embeddings)[0]
    # Get indices of the top k records
    idxs = np.argsort(-scores)[:top_k]

    print(f"Query: {query}")
    for idx in idxs:
        print(f"Score: {scores[idx]:.4f}")
        print(texts[idx])
        print("--------")

In [20]:
query("Give me some tips to improve my mental health.")

Query: Give me some tips to improve my mental health.
Score: 0.6206
Key Tips & Guidelines:

* Follow a balanced diet rich in fruits, vegetables, lean proteins, whole grains, and omega-3 fatty acids.
* Exercise consistently, aiming for at least 30 minutes per day most days of the week.
* Get sufficient sleep (7-9 hours nightly). Develop a bedtime routine and create a conducive sleeping environment.

Step 4: Cultivate Positive Relationships
Social connections play a vital role in promoting mental health. Building strong, supportive networks encourages open communication, empathy, and validation while combating loneliness and isolation.

Key Tips & Guidelines:

* Nurture existing friendships by scheduling regular meetups and engaging in shared interests.
* Seek out new social opportunities through clubs, classes, or volunteering.
* Learn to identify unhealthy relationship patterns and establish boundaries accordingly.

Step 5: Manage Stress Effectively
Chronic stress can wreak havoc on me

In [21]:
query_emb = query("Give me some tips for writing good code.")

Query: Give me some tips for writing good code.
Score: 0.5894
Step 6: Improve Code Quality
Strive for clean, readable, maintainable code. Adopt consistent naming conventions, indentation styles, and formatting rules. Utilize version control systems like Git to track changes and collaborate effectively. Leverage linters and static analyzers to enforce style guides automatically. Document your work using comments and dedicated documentation tools. High-quality code facilitates collaboration, promotes longevity, and simplifies troubleshooting.

Step 7: Embrace Best Practices
Follow established best practices relevant to your chosen language and domain. Examples include Object-Oriented Design Principles, SOLID principles, Test-Driven Development (TDD), Dependency Injection, Asynchronous Programming, etc. While seemingly overwhelming initially, integrating them gradually enhances design patterns, scalability, and extensibility. Consult authoritative blogs, books, and articles to stay update

In [22]:
query("How do I create a basic webpage?")

Query: How do I create a basic webpage?
Score: 0.6989
Creating a webpage with basic HTML is an exciting endeavor that allows you to share information, showcase your creativity, or build the foundation for a more complex website. This comprehensive guide will walk you through each step required to create a simple yet effective static webpage using HTML (HyperText Markup Language), the standard language used to structure content on the World Wide Web.

**Step 1: Choose a Text Editor**

Before diving into coding, select a suitable text editor for writing your HTML code. Popular choices include Notepad++ (Windows), Visual Studio Code (cross-platform), Atom (cross-platform), and Sublime Text (cross-platform). These editors offer features such as syntax highlighting, auto-completion, and error detection, making them ideal for beginners and professionals alike.

Key tip: Avoid using word processors like Microsoft Word or Google Docs, as they may add unnecessary formatting codes that can inter

In [23]:
query(
    "What are some environment-friendly practices I can incorporate in everyday life?"
)

Query: What are some environment-friendly practices I can incorporate in everyday life?
Score: 0.6416
* **Carry Reusable Bags**: Keep cloth or other reusable shopping bags handy for grocery trips and errands. Store extra bags in your car, purse, or backpack for convenience.
* **Choose Sustainable Alternatives**: Opt for reusable water bottles, coffee cups, straws, and cutlery made from eco-friendly materials like stainless steel, glass, silicone, or bamboo.
* **Support Businesses that Minimize Packaging**: Patronize establishments that prioritize sustainable practices, such as using biodegradable packaging or offering package-free options.

Step 3: Conserve Energy and Water
Explanation: Efficient energy and water usage help decrease reliance on nonrenewable resources and lower greenhouse gas emissions.
--------
Score: 0.6297
* Use public transportation, carpool, bike, or walk instead of driving alone.
* Switch to energy-efficient appliances and light bulbs.
* Unplug electronics when th